## Step1 : Preprocessing

In [75]:
# Import our dependencies
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import pandas as pd


In [76]:


#  Import and read the charity_data.csv.

application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [77]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#application_df.drop('EIN', axis=1)
application_df1 = application_df.drop('NAME', axis=1)
application_df2 = application_df1.drop('EIN', axis=1)
application_df2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [78]:
# Determine the number of unique values in each column.

application_df2.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [79]:
# Look at APPLICATION_TYPE value counts for binning
application_df2["APPLICATION_TYPE"].value_counts()



T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [80]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_type_cutoff_value = 528
application_types_to_replace = []
for idx,name in enumerate(application_df2["APPLICATION_TYPE"].value_counts().index.tolist()):
    if application_df2["APPLICATION_TYPE"].value_counts()[idx] < application_type_cutoff_value:
         application_types_to_replace.append(name)
# Replace in dataframe
for app in application_types_to_replace:
    application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [81]:
# Look at CLASSIFICATION value counts for binning
application_df2["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [82]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_vc = application_df2["CLASSIFICATION"].value_counts()
classification_vc[classification_vc > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [83]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = []
classifications_cutoff_value = 1883
for idx,name in enumerate(application_df2["CLASSIFICATION"].value_counts().index.tolist()):
    if application_df2["CLASSIFICATION"].value_counts()[idx] < classifications_cutoff_value:
         classifications_to_replace.append(name)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df2['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [84]:
# Convert categorical data to numeric with `pd.get_dummies`
data=pd.get_dummies(application_df2)
data

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [85]:
# Split our preprocessed data into our features and target arrays
y = data['IS_SUCCESSFUL'].values
X = data.drop(['IS_SUCCESSFUL'], axis = 1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [86]:
# Create a StandardScaler instances
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Step 2 : Compile, Train and Evaluate the Model

## Attempt 1

In [87]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
start = len(X_train[0])

h1 = 80
h2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(input_dim=start, units=h1, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=h2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 80)                3520      
_________________________________________________________________
dense_19 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [88]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [89]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 737us/step - loss: 43577.0224 - accuracy: 0.4984
Epoch 2/50
804/804 [==============================] - 1s 784us/step - loss: 46845.9525 - accuracy: 0.5077
Epoch 3/50
804/804 [==============================] - 1s 730us/step - loss: 61424.1416 - accuracy: 0.4856
Epoch 4/50
804/804 [==============================] - 1s 726us/step - loss: 17786.7926 - accuracy: 0.4835
Epoch 5/50
804/804 [==============================] - 1s 765us/step - loss: 33012.6000 - accuracy: 0.5028
Epoch 6/50
804/804 [==============================] - 1s 842us/step - loss: 30131.2203 - accuracy: 0.5102
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 59513.5685 - accuracy: 0.4880
Epoch 8/50
804/804 [==============================] - 1s 814us/step - loss: 14928.9903 - accuracy: 0.4917
Epoch 9/50
804/804 [==============================] - 1s 820us/step - loss: 28868.9271 - accuracy: 0.5056
Epoch 10/50
804/804 [===========================

In [90]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8984 - accuracy: 0.5174
Loss: 0.8984459042549133, Accuracy: 0.5174344182014465


## Attempt 2

In [65]:
attempt2_df = application_df
attempt2_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [91]:
attempt2_df1 = attempt2_df.drop(columns=["EIN", "NAME", "ORGANIZATION"], axis=1)
attempt2_df1.head()
attempt2_df1["INCOME_AMT"].value_counts()
attempt2_df1.describe()

,STATUS,ASK_AMT,IS_SUCCESSFUL
count,34299.000000,3.429900e+04,34299.000000
mean,0.999854,2.769199e+06,0.532406
std,0.012073,8.713045e+07,0.498956
min,0.000000,5.000000e+03,0.000000
25%,1.000000,5.000000e+03,0.000000
50%,1.000000,5.000000e+03,1.000000
75%,1.000000,7.742000e+03,1.000000
max,1.000000,8.597806e+09,1.000000


In [92]:
 # Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_type_cutoff_value = 156
application_types_to_replace = []
for idx,name in enumerate(attempt2_df1["APPLICATION_TYPE"].value_counts().index.tolist()):
    if attempt2_df1["APPLICATION_TYPE"].value_counts()[idx] < application_type_cutoff_value:
         application_types_to_replace.append(name)
# Replace in dataframe
for app in application_types_to_replace:
    attempt2_df1['APPLICATION_TYPE'] = attempt2_df1['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
attempt2_df1['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [93]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = []
classifications_cutoff_value = 777
for idx,name in enumerate(attempt2_df1["CLASSIFICATION"].value_counts().index.tolist()):
    if attempt2_df1["CLASSIFICATION"].value_counts()[idx] < classifications_cutoff_value:
         classifications_to_replace.append(name)

# Replace in dataframe
for cls in classifications_to_replace:
    attempt2_df1['CLASSIFICATION'] = attempt2_df1['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
attempt2_df1['CLASSIFICATION'].value_counts()



C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [94]:
data_attempt2=pd.get_dummies(attempt2_df1)
data_attempt2

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [95]:
# Split our preprocessed data into our features and target arrays
y_attempt2 = data_attempt2["IS_SUCCESSFUL"].values
X_attempt2 = data_attempt2.drop(["IS_SUCCESSFUL"], axis = 1).values

# Split the preprocessed data into a training and testing dataset
X_train_attempt2, X_test_attempt2, y_train_attempt2, y_test_attempt2 = train_test_split(X_attempt2, y_attempt2)

In [96]:
# Create a StandardScaler instances
scaler_attempt2 = StandardScaler()

# Fit the StandardScaler
X_scaler_attempt2 = scaler_attempt2.fit(X_train_attempt2)

# Scale the data
X_train_scaled_attempt2 = X_scaler_attempt2.transform(X_train_attempt2)
X_test_scaled_attempt2 = X_scaler_attempt2.transform(X_test_attempt2)

In [97]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
start = len(X_train_attempt2[0])
h1_attempt2 = 80
h2_attempt2 = 30

nn_attempt2 = tf.keras.models.Sequential()

# First hidden layer
nn_attempt2.add(tf.keras.layers.Dense(input_dim=start, units=h1_attempt2, activation="relu"))

# Second hidden layer
nn_attempt2.add(tf.keras.layers.Dense(units=h2_attempt2, activation="relu"))

# Output layer
nn_attempt2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_attempt2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 80)                3360      
_________________________________________________________________
dense_22 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 31        
Total params: 5,821
Trainable params: 5,821
Non-trainable params: 0
_________________________________________________________________


In [98]:
# Compile the model
nn_attempt2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_attempt2 = nn_attempt2.fit(X_train_attempt2,y_train_attempt2,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 790us/step - loss: 92053.7682 - accuracy: 0.4967
Epoch 2/50
804/804 [==============================] - 1s 774us/step - loss: 69856.8472 - accuracy: 0.4790
Epoch 3/50
804/804 [==============================] - 1s 723us/step - loss: 30339.1272 - accuracy: 0.4769
Epoch 4/50
804/804 [==============================] - 1s 728us/step - loss: 159680.2600 - accuracy: 0.5124
Epoch 5/50
804/804 [==============================] - 1s 730us/step - loss: 117802.3093 - accuracy: 0.4712
Epoch 6/50
804/804 [==============================] - 1s 724us/step - loss: 13829.1596 - accuracy: 0.4974
Epoch 7/50
804/804 [==============================] - 1s 731us/step - loss: 58504.0595 - accuracy: 0.5208
Epoch 8/50
804/804 [==============================] - 1s 731us/step - loss: 27623.3762 - accuracy: 0.5150
Epoch 9/50
804/804 [==============================] - 1s 750us/step - loss: 35034.5387 - accuracy: 0.5231
Epoch 10/50
804/804 [=======================

In [99]:
# Evaluate the model using the test data
model_loss_attempt2, model_accuracy_attempt2 = nn_attempt2.evaluate(X_test_scaled_attempt2,y_test_attempt2,verbose=2)
print(f"Loss: {model_loss_attempt2}, Accuracy: {model_accuracy_attempt2}")

268/268 - 0s - loss: 0.8510 - accuracy: 0.6833
Loss: 0.85103839635849, Accuracy: 0.6832653284072876


## Attempt 3

In [100]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
start = len(X_train_attempt2[0])
h1_attempt3 = 30
h2_attempt3 = 25

nn_attempt3 = tf.keras.models.Sequential()

# First hidden layer
nn_attempt3.add(tf.keras.layers.Dense(input_dim=start, units=h1_attempt3, activation="relu"))

# Second hidden layer
nn_attempt3.add(tf.keras.layers.Dense(units=h2_attempt3, activation="relu"))

# Output layer
nn_attempt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_attempt3.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 30)                1260      
_________________________________________________________________
dense_25 (Dense)             (None, 25)                775       
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 26        
Total params: 2,061
Trainable params: 2,061
Non-trainable params: 0
_________________________________________________________________


In [101]:
# Compile the model
nn_attempt3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_attempt3 = nn_attempt3.fit(X_train_attempt2,y_train_attempt2,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 737us/step - loss: 38868.1686 - accuracy: 0.5026
Epoch 2/50
804/804 [==============================] - 1s 724us/step - loss: 50808.6074 - accuracy: 0.4849
Epoch 3/50
804/804 [==============================] - 1s 719us/step - loss: 47019.9337 - accuracy: 0.4973
Epoch 4/50
804/804 [==============================] - 1s 739us/step - loss: 15923.0891 - accuracy: 0.5072
Epoch 5/50
804/804 [==============================] - 1s 750us/step - loss: 5780.3452 - accuracy: 0.4996
Epoch 6/50
804/804 [==============================] - 1s 802us/step - loss: 14925.2975 - accuracy: 0.4943
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 52453.8375 - accuracy: 0.4865
Epoch 8/50
804/804 [==============================] - 1s 832us/step - loss: 12856.8628 - accuracy: 0.4857
Epoch 9/50
804/804 [==============================] - 1s 835us/step - loss: 19073.6372 - accuracy: 0.4821
Epoch 10/50
804/804 [============================

In [102]:
# Evaluate the model using the test data
model_loss_attempt3, model_accuracy_attempt3 = nn_attempt3.evaluate(X_test_scaled_attempt2,y_test_attempt2,verbose=2)
print(f"Loss: {model_loss_attempt3}, Accuracy: {model_accuracy_attempt3}")

268/268 - 0s - loss: 0.8741 - accuracy: 0.5775
Loss: 0.8741198778152466, Accuracy: 0.5774927139282227


In [103]:
nn_attempt3.save("AlphabetSoupCharity_Optimization.h5")